In [4]:
class Animal:
    def __init__(self, name, species):
        self.name = name
        self.species = species

    def make_sound(self):
        print(f"{self.name} makes a sound.")

class Dog(Animal):
    def __init__(self, name, species, breed):
        self.name = name      # 부모 클래스와 동일한 이름의 속성 재정의
        self.species = species
        self.breed = breed

    def make_sound(self):
        print(f"{self.name} barks.")

# 인스턴스 생성
my_dog = Dog(name="Buddy", species="Dog", breed="Golden Retriever")

print(f"Name: {my_dog.name}")     # 출력: Name: Buddy
print(f"Species: {my_dog.species}") # 출력: Species: Dog
print(f"Breed: {my_dog.breed}")     # 출력: Breed: Golden Retriever

my_dog.make_sound()  # 출력: Buddy barks.


Name: Buddy
Species: Dog
Breed: Golden Retriever
Buddy barks.


In [ ]:
import torch

# MPS(Apple Silicon GPU) 사용 가능 여부 체크
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Using device:", device)

# DeZero Variable 클래스와 유사한 PyTorch 기반 클래스
class Variable:
    def __init__(self, data):
        if not isinstance(data, torch.Tensor):
            data = torch.tensor(data, dtype=torch.float32)
        self.data = data.to(device)
        self.grad = None

    def __repr__(self):
        return f"Variable({self.data}, grad={self.grad})"

# DeZero Function과 유사한 클래스
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

# Square 함수 구현
class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.input.data
        return 2 * x * gy

# Exp 함수 구현
class Exp(Function):
    def forward(self, x):
        return torch.exp(x)

    def backward(self, gy):
        x = self.input.data
        return torch.exp(x) * gy

# 수치 미분 함수
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

# 테스트
if __name__ == "__main__":
    A = Square()
    B = Exp()
    C = Square()

    x = Variable(torch.tensor(0.5))
    a = A(x)
    b = B(a)
    y = C(b)

    print("Forward result:", y.data)

    # 수치 미분으로 검증
    f = lambda var: C(B(A(var)))
    dy = numerical_diff(f, x)
    print("Numerical diff:", dy)

    print("Executed on:", device)


Using device: mps
Forward result: tensor(1.6487, device='mps:0')
Numerical diff: tensor(3.2979, device='mps:0')
Executed on: mps


In [2]:
import p1
print(__name__)

__main__


In [4]:
class MyClass:
    def __init__(self):
        self.__private_var = 42

    def __private_method(self):
        return "This is a private method"

    def public_method(self):
        return self.__private_method()          # _에 관한 접근 제어는 자바에서 공부하고 다시 공부하기로 한다!

obj = MyClass()
print(obj.public_method())  # This is a private method

# 클래스 외부에서 직접 접근 시도
print(obj.__init__)  # AttributeError: 'MyClass' object has no attribute '__private_var'
print(obj.__private_method())  # AttributeError: 'MyClass' object has no attribute '__private_method'

# 네임 맹글링을 통해 접근 시도 (비권장)
print(obj._MyClass__private_var)  # 42
print(obj._MyClass__private_method())  # This is a private method


This is a private method
<bound method MyClass.__init__ of <__main__.MyClass object at 0x1042ab410>>


AttributeError: 'MyClass' object has no attribute '__private_method'

In [7]:
# current_module.py
print("__name__ in current_module:", __name__)

from and_gate import *

# and_gate.py
print("__name__ in and_gate:", __name__)


__name__ in current_module: __main__
__name__ in and_gate: __main__
